# Understanding Aqua's Operator Flow: A Lingua Franca Between Quantum Algorithms Theory and Engineering

_donny@, 30-Apr-20_

## Overview and Design Philosophy

A library for Quantum Algorithms & Applications is more than a collection of procedural code wrapped in Python functions. It needs to provide tools to make writing algorithms simple and easy. This is the layer of modules between the circuits and algorithms, providing the language and computational primitives for QA&A research.

In Aqua, we call this layer the Operator Flow. It works by unifying computation with theory through the common language of functions and operators, in a way which preserves physical intuition and programming freedom. In the Operator Flow, we construct functions over binary variables, manipulate those functions with operators, and evaluate properties of these functions. So far, that can all be done classically with probabilistic or even deterministic programming, but you'll see that often the most efficient way to represent these interactions compactly is using quantum circuits. The illuminating examples here will often be classic primitives of quantum computation, such as Hamiltonian simulation or the Quantum Fourier Transform.

Below, we'll describe the key players in the Operator Flow, and show some examples of how they can be used to construct some familiar quantum algorithms. This notebook assumes a basic familiarity with Quantum Algorithms.

### North Star

The Operator Flow is meant to serve as a lingua franca between the theory and implementation of Quantum Algorithms & Applications. Meaning, the ultimate goal is that when theorists speak their theory in the Operator Flow, they are speaking valid implementation, and when the engineers speak their implementation in the Operator Flow, they are speaking valid physical formalism. To be successful, it must be fast and physically formal enough for theorists to find it easier and more natural than hacking Matlab or NumPy, and the engineers must find it straightforward enough that they can learn it as a typical software library, and learn the physics naturally and effortlessly as they learn the code. There can never be a point where we say "below this level this is all hacked out, don't come down here, stay in the interface layer above." It all must be clear and learnable.

### Basic Definitions

Before getting into the details of the code, it's important to note that three mathematical concepts unpin the Operator Flow. We derive most of the inspiration for the code structure from [John Watrous's formalism](https://cs.uwaterloo.ca/~watrous/TQI/) (but do not follow it exactly), so it may be worthwhile to review Chapters I and II, which are free online, if you feel the concepts are not clicking.

1. An $n-qubit State function$ is a complex function over $n$ binary variables, which we will often refer to as _n-qubit binary strings_. For example, the traditional quantum "zero state" is a 1-qubit state function, with a definition of $f(0) = 1$ and $f(1) = 0$.
1. An $n-qubit Operator$ is a linear function taking n-qubit state functions to n-qubit state functions. For example, the Pauli X Operator is defined by $f(Zero) = One$ and $f(One) = Zero$. Equivalently, an Operator can be defined as a complex function over two n-qubit binary strings, and it is sometimes convenient to picture things this way. By this definition, our Pauli X can be defined by its typical matrix elements, $f(0, 0) = 0$, $f(1, 0) = 1$, $f(0, 1) = 1$, $f(1, 1) = 0$.
1. An $n-qubit Measurement$ is a functional taking n-qubit State functions to complex values. For example, a Pauli Z Measurement can be defined by $f(Zero) = 0$ and $f(One) = 1$.

### High Level Object Structure

The Operator Flow includes two primary groups of actors: 
* Operators, objects which represent functions or functionals, all deriving from `OperatorBase`: 
    * `PrimitiveOp`s - Operators whose behavior are defined by some internal computational primitive from Terra, such as a Terra's `Pauli` (`PauliOp`), `Operator` (`MatrixOp`), or `QuantumCircuit` (`CircuitOp`).
    * `StateFn`s - A dual class for representing State functions and Measurements, the behavior of which is defined by some internal primitive, e.g. a simple dict (`DictStateFn`), Terra's `Statevector` (`VectorStateFn`), a `QuantumCircuit` (`CircuitStateFn`), or an OperatorBase representing a density operator (`OperatorStateFn`).
    * `ListOp`s - Containers of PrimitiveOps or StateFns with a particular combination logic to be evaluated later, such as addition (`SummedOp`), composition (`ComposedOp`), tensor product (`TensoredOp`), and concatination into a list (`ListOp`).
    * `EvolvedOp` - A special case of a `PrimitiveOp` holding an `Operatorbase` as its primitive, serving as a placeholder for an evolution algorithm to convert later into an Operator approximating the exponentiation of the `OperatorBase`.
* Converters, objects which manipulate Operators, all deriving from `Converter`:
    * `ExpectationValue`s - Traverse over an Operator and replace OperatorStateFn measurements into Z-basis or matrix measurements approximating the expectation value of the measurement.
    * `Evolution`s - Traverse over an Operator and replace `EvolvedOp`s with circuits approximating the evolution.
    * `CircuitSampler`s - Traverse over an Operator and replace `CircuitStateFn`s with `DictStateFn`s approximating them, perhaps sampled from a quantum device.
    * Other converters, e.g. AbelianGrouper, PauliBasisChange
    
Don't worry if this seems like a whirlwind. We'll discuss each of these in detail in the next section.

## Part I: State Functions and Measurements

The most basic unit of computation in the Operator Flow is the state function, abbreviated as StateFn, a simple complex function over binary variables. There are several StateFn instances built in for convenience, `Zero, One, Plus, Minus`, and four ways a StateFn can be defined, `DictStateFn, VectorStateFn, CircuitStateFn, OperatorStateFn`.

In [1]:
from qiskit.aqua.operators import (StateFn, Zero, One, Plus, Minus, 
                                   DictStateFn, VectorStateFn, CircuitStateFn, OperatorStateFn)

The function this object represents, mapping a binary string to a complex value, can be accessed via the `.eval` method.

In [2]:
print(Zero)

DictStateFn({'0': 1})


In [3]:
print(Zero.eval('0'))
print(Zero.eval('1'))
print(One.eval('1'))
print(Plus.eval('0'))
print(Minus.eval('1'))

1.0
0.0
1.0
(0.70710678118655+0j)
(-0.70710678118655+0j)


This should look familiar as something analogous to the quantum state or wavefunctions you know, $|0\rangle$, $|1\rangle$, $|+\rangle$, and $|-\rangle$, but with a key difference: these statefunctions need not be normalized. `statefunction1 + statefunction2` behaves exactly as you'd expect mathematical functions to behave, producing a new function whose value for a given binary string is the sum of the corresponding values for a different function.

In [4]:
print((One + One + One).eval('1'))
print((Plus + Minus).eval('0'))
print((Plus + Minus).eval('1'))

3.0
(1.4142135623731+0j)
0j


The behavior of each StateFn type is defined internally by some data structure, which we call the primitive, and a complex coefficient. The `DictStateFn` is the simplest type, holding a dict primitive of {string: complex} value pairs. `Zero` and `One` are simply these. Any value missing from the dict is simply equal to 0.

In [12]:
print(Zero)
print(One)
print(Zero.primitive)
print(Zero.coeff)
print((Zero + Zero + Zero).coeff)
print((Zero * 2j).coeff)

DictStateFn({'0': 1})
DictStateFn({'1': 1})
{'0': 1}
1.0
3.0
2j


For simplicity, the `StateFn` class also doubles as the class for Measurement. The adjoint of a `StateFn` is simply a measurement defined by the same primitive as the state. This is conceptually identical to the idea that a bra is the adjoint of a ket, and it is convenient to think of it that way.

Just as the State function over binary variables is accessible via `.eval`, so to the Measurement functional over State functions is available via `.eval`. As we'll see below, the availability of the underlying function through `.eval` is also true for the Operators.

In [6]:
print(One.adjoint())
print(Zero.adjoint().eval(One))
print(Zero.adjoint().eval(Zero))
print(Zero.adjoint().eval(Plus))

DictMeasurement({'1': 1})
0.0
1.0
(0.70710678118655+0j)


Note that we can also perform function composition between a State function and measure using `.compose`, and `.eval` with no argument later.

In [10]:
print(Zero.adjoint().compose(One).eval())
print(Zero.adjoint().compose(Zero).eval())
print(Zero.adjoint().compose(Plus).eval())

0.0
1.0
(0.70710678118655+0j)


Nearly all arithmetic operations between StateFns are supported, including:
* `+` - addition
* `-` - subtraction, negation (scalar multiplication by -1)
* `*` - scalar multiplication
* `\` - scalar division
* `@` - composition
* `^` - tensor product or tensor power (tensor with self n times)
* `**` - composition power (compose with self n times)
* `==` - equality
* `~` - adjoint, alternating between a State Function and Measurement

This arithmetic, along with the Operators, allows quick and easy construction of many states for Quantum Algorithms. However, **one must be careful to use parentheses properly when performing arithmetic in this way.**

In [7]:
print(600 * ((One^5) + (Zero^5)))
print(One^Zero^3)

DictStateFn({'11111': 1.0, '00000': 1.0}) * 600.0
DictStateFn({'101010': 1})


States can also be easily converted between primitive. Below we'll see that we can also represent State functions by vectors and Quantum circuits.

In [8]:
print(((One^5) + (Zero^5)).to_matrix_op())
print(((Plus^Minus^Plus^Minus)^2).to_circuit_op())

VectorStateFn(Statevector([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
            dims=(2, 2, 2, 2, 2)))
CircuitStateFn(
     ┌───┐┌───┐
q_0: ┤ X ├┤ H ├
     ├───┤└───┘
q_1: ┤ H ├─────
     ├───┤┌───┐
q_2: ┤ X ├┤ H ├
     ├───┤└───┘
q_3: ┤ H ├─────
     ├───┤┌───┐
q_4: ┤ X ├┤ H ├
     ├───┤└───┘
q_5: ┤ H ├─────
     ├───┤┌───┐
q_6: ┤ X ├┤ H ├
     ├───┤└───┘
q_7: ┤ H ├─────
     └───┘     
)


In fact, `Plus` and Minus are `CircuitStateFn`s.

In [9]:
print(Plus)
print(Minus)

CircuitStateFn(
     ┌───┐
q_0: ┤ H ├
     └───┘
)
CircuitStateFn(
     ┌───┐┌───┐
q_0: ┤ X ├┤ H ├
     └───┘└───┘
)


In [1]:
import numpy as np
from qiskit.aqua.operators import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution
from qiskit.circuit import Parameter

## Construct an H2 Hamiltonian

In [2]:
two_qubit_H2 =  (-1.0523732 * I^I) + \
                (0.39793742 * I^Z) + \
                (-0.3979374 * Z^I) + \
                (-0.0112801 * Z^Z) + \
                (0.18093119 * X^X)

## Evolve a Bell state by Our Hamiltonian

OpFlow fully supports parameterization, so we can use a parameter for our evolution time here. Notice that there's no "evolution time" argument in any function. OpFlow exponentiates whatever operator we tell it to, and if we choose to multiply the operator by an evolution time, $e^{iHt}$, that will be refected in our exponentiation parameters. This is not some trick to make it look like Physics - it actually works this way under the hood.

In [21]:
# Meaningless state
bell = CX @ (H^I) @ Zero
# We can also do CX @ (Plus ^ Zero)
evo_time = Parameter('θ')
wf = (evo_time*two_qubit_H2).exp_i() @ bell
trot = PauliTrotterEvolution(trotter_mode='trotter').convert(wf)
trot.to_circuit().draw(fold=1000)
# trot.to_circuit().data[0]

┌───┐┌───┐┌──────────────────┐┌───┐┌───┐┌───┐┌──────────────────┐┌───┐┌──────────────────┐
q_0: ───────■──┤ H ├┤ X ├┤ Rz(0.18093119*θ) ├┤ X ├┤ H ├┤ X ├┤ Rz(-0.0112801*θ) ├┤ X ├┤ Rz(0.39793742*θ) ├
     ┌───┐┌─┴─┐├───┤└─┬─┘└──────────────────┘└─┬─┘├───┤└─┬─┘└──────────────────┘└─┬─┘├──────────────────┤
q_1: ┤ H ├┤ X ├┤ H ├──■────────────────────────■──┤ H ├──■────────────────────────■──┤ Rz(-0.3979374*θ) ├
     └───┘└───┘└───┘                              └───┘                              └──────────────────┘

We can bind our parameter to the operator if we so choose, and it will recursively bind into the circuit.

In [9]:
bound = trot.bind_parameters({evo_time: .5})
bound.to_circuit().draw(fold=1000)

┌───┐┌───┐┌────────────────────────┐┌───┐┌───┐┌───┐┌──────────────────────────┐┌───┐┌───────────────────────┐ 
q_0: ───────■──┤ H ├┤ X ├┤ Rz(0.0904655950000000) ├┤ X ├┤ H ├┤ X ├┤ Rz(-0.00564005000000000) ├┤ X ├┤ Rz(0.198968710000000) ├─
     ┌───┐┌─┴─┐├───┤└─┬─┘└────────────────────────┘└─┬─┘├───┤└─┬─┘└──────────────────────────┘└─┬─┘├───────────────────────┴┐
q_1: ┤ H ├┤ X ├┤ H ├──■──────────────────────────────■──┤ H ├──■────────────────────────────────■──┤ Rz(-0.198968700000000) ├
     └───┘└───┘└───┘                                    └───┘                                      └────────────────────────┘

## Now that we have a state, let's measure the energy of the state

In [10]:
from qiskit import BasicAer
backend = BasicAer.get_backend('qasm_simulator')

PauliExpectation(operator=two_qubit_H2, backend=backend).compute_expectation(bound)

(-0.6454481153515621+0j)

### Why We Call it OpFlow

The power of OpFlow is that everything is an Operator, so we can ask PauliExpectation for the intermediate operator it generated before it used a Circuit sampler to replace the circuits with dicts:

In [11]:
expect_op = PauliExpectation(operator=two_qubit_H2, backend=backend).expectation_op(bound)
print(expect_op)

SummedOp(
[ComposedOp(
[Measurement(SummedOp(
[0.18093119 * ZZ,
-1.0523732 * II])),
StateFunction(
               ┌───┐┌───┐┌────────────────────────┐┌───┐┌───┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ Rz(0.0904655950000000) ├┤ X ├┤ H ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└────────────────────────┘└─┬─┘├───┤└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■──────────────────────────────■──┤ H ├──■──»
     └───┘└───┘└───┘                                    └───┘     »
«     ┌──────────────────────────┐┌───┐┌───────────────────────┐ ┌───┐
«q_0: ┤ Rz(-0.00564005000000000) ├┤ X ├┤ Rz(0.198968710000000) ├─┤ H ├
«     └──────────────────────────┘└─┬─┘├───────────────────────┴┐├───┤
«q_1: ──────────────────────────────■──┤ Rz(-0.198968700000000) ├┤ H ├
«                                      └────────────────────────┘└───┘
)]),
ComposedOp(
[Measurement(SummedOp(
[0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ])),
StateFunction(
               ┌───┐┌───┐┌────────────────────────┐┌───┐┌───┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ R

### We can just as easily take the Expectation over a _vector_ of Pauli Operators.

In [16]:
ham_list = ListOp([X^X, Y^Y, Z^Z, two_qubit_H2])
expect = PauliExpectation(operator=ham_list, backend=backend).compute_expectation(bound)
expect

[(0.01953125+0j),
 (0.013671874999999917+0j),
 (0.060546875+0j),
 (-0.6361677128906251+0j)]

### We Can Even Take the Expectation of a vector of Observables over a vector of StateFns

In [17]:
# Here we're using PauliExpectation's param argument instead of passing the bound OpVec below in case we can 
# take advantage of late binding / parameterized Qobj
params = {evo_time: [.5, 1.0, 1.5]}
expects = PauliExpectation(operator=ham_list, backend=backend).compute_expectation(trot, params=params)
np.real(np.around(expects, decimals=3))

array([[ 0.02 ,  0.006, -0.018, -0.64 ],
       [ 0.094,  0.062, -0.029, -0.661],
       [ 0.145,  0.07 ,  0.014, -0.632]])

### Parameter binding also supports binding lists

In [18]:
print(trot.bind_parameters(params))

ListOp(
[StateFunction(
               ┌───┐┌───┐┌────────────────────────┐┌───┐┌───┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ Rz(0.0904655950000000) ├┤ X ├┤ H ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└────────────────────────┘└─┬─┘├───┤└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■──────────────────────────────■──┤ H ├──■──»
     └───┘└───┘└───┘                                    └───┘     »
«     ┌──────────────────────────┐┌───┐┌───────────────────────┐ 
«q_0: ┤ Rz(-0.00564005000000000) ├┤ X ├┤ Rz(0.198968710000000) ├─
«     └──────────────────────────┘└─┬─┘├───────────────────────┴┐
«q_1: ──────────────────────────────■──┤ Rz(-0.198968700000000) ├
«                                      └────────────────────────┘
),
StateFunction(
               ┌───┐┌───┐┌───────────────────────┐┌───┐┌───┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ Rz(0.180931190000000) ├┤ X ├┤ H ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└───────────────────────┘└─┬─┘├───┤└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■─────────────────────────────■──┤ H ├──■──»
     └───┘└───┘└───┘